# PNOFi

![DoNOF](https://donof.readthedocs.io/en/latest/_images/Logo-DoNOF.jpeg)

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

En este notebook se encuentra el álgebra elemental para calcular un punto simple de energía de PNOFi (i=5,6,7)

**Seleccionamos un PNOFi (i=5,7)** y el tipo de gradiente, $\frac{dE}{dn_i}$, para los n.umeros de ocupación.

In [1]:
PNOFi = 7
gradient = "numerical" # analytical/numerical

## Introducción

**Importamos librerías**

In [2]:
import psi4
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import eigh
from time import time

**Seleccionamos una molécula**, y otros datos como la memoria del sistema y la base

In [3]:
psi4.set_memory('4 GB')

mol = psi4.geometry("""
O  0.0000   0.000   0.116
H  0.0000   0.749  -0.453
H  0.0000  -0.749  -0.453
  symmetry c1
""")

psi4.set_options({'basis': 'cc-pVDZ',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})

Construimos la función de onda y **evaluamos matrices e integrales en orbital atómico**, $S$, $T$, $V$, $H$, $(\mu\nu|\sigma\lambda)$

In [4]:
# Wavefunction
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))

# Integrador
mints = psi4.core.MintsHelper(wfn.basisset())

# Overlap, Kinetics, Potential
S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V

# Integrales de Repulsión Electrónica, ERIs (mu nu | sigma lambda)
I = np.asarray(mints.ao_eri())

# Energía Nuclear
E_nuc = mol.nuclear_repulsion_energy()

**Se declaran variables del sistema.**

El número de electrones (ne) es la suma de los electrones con spín alpha (nalpha) y los electrones con spín beta (nbeta)
\begin{equation}
N_e = N_{\alpha} + N_{\beta}
\end{equation}

El número de orbitales doblemente ocupados inactivos (no1), es decir con ocupación de 1, está dado por
\begin{equation}
N_{o1} = \left\{
  \begin{array}{lll}
  \sum_{átomo}^{N_{átomos}} {orbitales\ de\ core}_{átomo}      & Default \\
  Otro     & \mathrm{si\ se\ indica}
  \end{array}
  \right.
\end{equation}

*****************************************************************************************************************

Dividiremos el espacio orbital en dos subespacios
\begin{equation}
\Omega = \Omega_I + \Omega_{II}
\end{equation}

*****************************************************************************************************************

El subespacio $\Omega_{II}$ se divide en $N_{II}/2$ subespacios $\Omega_g$. Cada subespacio $\Omega_g \in \Omega_{II}$ contiene un orbital $\ket{g}$ con $g \le N_{II}/2$ y $N_g$ orbitales $\ket{p}$ con $p > N_{II}/2$, es decir
\begin{equation}
\Omega_g = \{\ket{g},\ket{p_1},\ket{p_2},\cdots,\ket{p_{N_{g}}}\} 
\end{equation}

El número de orbitales fuertemente doble ocupados (ndoc) está dado por
\begin{equation}
N_{doc} = N_{\beta} - N_{o1} = N_{II}/2
\end{equation}

*****************************************************************************************************************

El subespacio $\Omega_I$ se compone por $N_I$ subespacios $\Omega_g$, en este caso

El número de orbitales fuertemente ocupados con ocupación simple (nsoc) está dado por
\begin{equation}
N_{soc} = N_{\alpha} - N_{\beta} = N_I
\end{equation}

*****************************************************************************************************************

El número de orbitales fuertemente ocupados (ndns) está dado por
\begin{equation}
N_{ndns} = N_{doc} + N_{soc} = N_{II}/2 + N_I = N_{\Omega}
\end{equation}

El número de orbitales virtuales (nvir) es la diferencia entre los electrones $N_\alpha$ (nalpha) y el número de funciones de base $N_{bf}$ 
\begin{equation}
N_{vir} = N_{bf} - N_{\alpha}
\end{equation}

El número de orbitales débilmente ocupados acoplados (ncwo) a cada orbital fuertemente doble ocupado (ndoc), y que constituye $N_g$, está dado por
\begin{equation}
N_{cwo} = \left\{
  \begin{array}{lll}
  N_{vir}/N_{doc}      & \mathrm{si\ } N_e = 2 \\
  1 & \mathrm{si\ } N_e > 2 \mathrm{\ ó\ } N_{cwo}>N_{vir}/N_{doc}\\
  Otro     & \mathrm{si\ se\ indica}
  \end{array}
  \right.
\end{equation}

*****************************************************************************************************************

La dimensión del subespacio de orbitales activos está dada por los orbitales fuertemente doble ocupados y sus orbitales débilmente ocupados asociados
\begin{equation}
N_{ac} = N_{doc} + N_{doc} N_{cwo}
\end{equation}

*****************************************************************************************************************

Los orbitals con número de ocupación distinto de cero (nbf5) son
\begin{equation}
N_{bf5} = N_{o1} + N_{ac} + N_{soc} 
\end{equation}
es decir, la suma de los orbitales de core (no1),más los fuertemente doble ocupados (ndoc) con sus orbitales débilmente ocupados acoplados (nwco) y los orbitales fuertemente ocupados con ocupación simple (nsoc)
\begin{equation}
N_{bf5} = N_{o1} + N_{doc} + N_{doc}N_{wco} + N_{soc}
\end{equation}

*****************************************************************************************************************

Establecemos los orbitales a optimizar (noptorb)
\begin{equation}
N_{optorb} = \left\{
  \begin{array}{lll}
  N_{bf}      & Default \\
  Otro     & \mathrm{si\ se\ indica}
  \end{array}
  \right.
\end{equation}

In [5]:
natoms = mol.natom()
nbf = S.shape[0]
nalpha = wfn.nalpha()
nbeta = wfn.nbeta()
ne = nalpha + nbeta
mul = mol.multiplicity()
no1 = 0 #Number of inactive doubly occupied orbitals | Se puede variar
for i in range(natoms):
    Z = mol.Z(i)
    if ( 1<=Z and Z<=  2):
        no1 += 0           # H-He
    elif ( 3<=Z and Z<= 10):
        no1 +=  1          # Li-Ne
    elif (11<=Z and Z<= 18):
        no1 +=  5          # Na-Ar
    elif(19<=Z and Z<= 36):
        no1 +=  9          # K-Kr
    elif(37<=Z and Z<= 49):
        no1 += 18          # Rb-In
    elif(50<=Z and Z<= 54):
        no1 += 23          # Sn-Xe
    elif(55<=Z and Z<= 71):
        no1 += 27          # Cs-Lu
    elif(72<=Z and Z<= 81):
        no1 += 30          # Hf-Tl
    elif(82<=Z and Z<= 86):
        no1 += 39          # Pb-Rn
    elif(87<=Z and Z<=109):
        no1 += 43          # Fr-Mt
    
ndoc = nbeta   -   no1
nsoc = nalpha  -   nbeta
ndns = ndoc    +   nsoc
nvir = nbf     -   nalpha

ncwo = 1
if(ne==2):
    ncwo= -1
if(ndns!=0):
    if(ndoc>0):
        if(ncwo!=1):
            if(ncwo==-1 or ncwo > nvir/ndoc):
                ncwo = int(nvir/ndoc)
    else:
        ncwo = 0

nac = ndoc * (1 + ncwo)
nbf5 = no1 + nac + nsoc   #JFHLY warning: nbf must be >nbf5
no0 = nbf - nbf5

noptorb = nbf     

closed = (nbeta == (ne+mul-1)/2 and nalpha == (ne-mul+1)/2)

Establecemos algunos parámetros

In [6]:
maxit = 1000  # Número máximo de iteraciones de Occ-SCF
no1 = no1     # Número de orbitales inactivos con ocupación 1
thresheid = 10**-6#8 # Convergencia de la energía total
maxitid = 30  # Número máximo de iteraciones externas
ipnof = 7     # PNOFi a calcular
threshl = 10**-3#4   # Convergencia de los multiplicadores de Lagrange
threshe = 10**-4#6   # Convergencia de la energía
threshec = 10**-10 # Convergencia  de la energía en optimización orbital
threshen = 10**-10 # Convergencia  de la energía en optimización de ocupaciones
scaling = True     # Scaling for f
nzeros = 1#0
nzerosm = 5
nzerosr = 2
itziter = 10        # Iteraciones para scaling constante
diis = True         # DIIS en optimización orbital
thdiis = 10**-3     # Para iniciar DIIS
ndiis = 5           # Número de ciclos para interpolar matriz de Fock generalizada en DIIS
perdiis = True      # Aplica DIIS cada NDIIS (True) o después de NDIIS (False) 
ncwo = ncwo         # Número de orbitales débilmente ocupados acoplados a cada orbital fueremtente ocupado 
noptorb = noptorb   # Número de orbitales a optimizar Nbf5 <= Noptorb <= Nbf
scaling = True
maxloop = 30

Obtenemos un Guess acorde a la ecuación

\begin{equation}
HC = SC\varepsilon
\end{equation}

In [7]:
E_i,C = eigh(H, S)

**Revisamos la ortonormalidad.** Asumiremos que los orbitales son ortonormales si $C^T SC$ difiere en menos de $10^{-6}$ respecto a la matriz identidad. Si los orbitales no son ortonormales, haremos hasta 3 intentos por ortonormalizarlos.

In [8]:
# Revisa ortonormalidad
orthonormality = True

CTSC = np.matmul(np.matmul(np.transpose(C),S),C)
ortho_deviation = np.abs(CTSC - np.identity(nbf))

if (np.any(ortho_deviation > 10**-6)):
    orthonormality = False

if not orthonormality:
    print("Orthonormality violations {:d}, Maximum Violation {:f}".format((ortho_deviation > 10**-6).sum(),ortho_deviation.max()))        
else:
    print("No violations of the orthonormality")

No violations of the orthonormality


Se cambian de fase los obitales moleculares tal que **en cada orbital sea positivo el coeficiente de mayor magnitud**

In [9]:
# Vuelve positivo el elemento más largo de cada MO
for j in range(nbf):
    
    #Obtiene el índice del coeficiente con mayor valor absoluto del MO
    idxmaxabsval = 0
    for i in range(nbf):
        if(abs(C[i][j])>abs(C[idxmaxabsval][j])):
            idxmaxabsval = i
    
    # Ajusta el signo del MO
    sign = np.sign(C[idxmaxabsval][j])
    C[0:nbf,j] = sign*C[0:nbf,j]

## Cálculo de orbitales y multiplicadores de Lagrange a partir de J, K y F

Crearemos una función que calcule

\begin{eqnarray}
D_{\mu\nu}^{(i)} &=& C_{\mu i} C_{\nu i}\\
J_{\mu\nu}^{(i)} &=& \sum_{\sigma\lambda} D_{\sigma\lambda}^{(i)} (\mu\nu|\sigma\lambda)\\
K_{\mu\sigma}^{(i)} &=& \sum_{\nu\lambda} D_{\nu\lambda}^{(i)} (\mu\nu|\sigma\lambda)
\end{eqnarray}

In [10]:
def computeJK(C):
    
    #denmatj    
    D = np.einsum('mi,ni->imn', C[:,0:nbf5], C[:,0:nbf5], optimize=True)
    
    #hstarj
    J = np.einsum('isl,mnsl->imn', D, I, optimize=True)    
    
    #hstark
    K = np.einsum('inl,mnsl->ims', D, I, optimize=True)    

    return J,K

Construimos

\begin{equation}
\lambda_{qp} = n_p H_{qp} + \int dr \frac{d V_{ee}}{d \phi_p (r)} \phi_q (r)
\end{equation}

Definimos una función para calcular la matriz generalizada de Fock
\begin{equation}
F_{\mu\nu}^{(i)} = \left\{
  \begin{array}{lll}
  H_{\mu\nu} + \sum_j^{N_{bf5}} J_{\mu\nu}^{(j)} C^{J}_{1j} - \sum_j^{N_{bf5}} K_{\mu\nu}^{(j)} C^{K}_{1j}      & \mathrm{si\ } i \in [1,N_{o1}]  \\
  n_{i}(H_{\mu\nu} + J_{\mu\nu}^{(i)}) + \sum_{j \ne i}^{N_{bf5}} J_{\mu\nu}^{(j)} C^{J}_{ij} - \sum_{j \ne i}^{N_{bf5}} K_{\mu\nu}^{(j)} C^{K}_{ij}     & \mathrm{si\ } i \in (N_{o1},N_{beta}]\\
  n_{i}H_{\mu\nu} + \sum_{j \ne i}^{N_{bf5}} J_{\mu\nu}^{(j)} C^{J}_{ij} - \sum_{j \ne i}^{N_{bf5}} K_{\mu\nu}^{(j)} C^{K}_{ij}     & \mathrm{si\ } i \in (N_{beta},N_{alpha}]\\
  n_i(H_{\mu\nu} + J_{\mu\nu}^{(i)}) + \sum_{j \ne i}^{N_{bf5}} J_{\mu\nu}^{(j)} C^J_{ij} - \sum_{j \ne i}^{N_{bf5}} K_{\mu\nu}^{(j)} C^K_{ij}     & \mathrm{si\ } i \in (N_{\alpha},N_{bf5}]
  \end{array}
  \right.
\end{equation}

In [11]:
def computeF(J,K,n,cj12,ck12):
    
    # Matriz de Fock Generalizada                    
    F = np.zeros((nbf5,nbf,nbf))    

    ini = 0
    if(no1>1):        
        ini = no1       
        
    # nH
    F += np.einsum('i,mn->imn',n,H,optimize=True)        # i = [1,nbf5]

    # nJ
    F[ini:nbeta,:,:] += np.einsum('i,imn->imn',n[ini:nbeta],J[ini:nbeta,:,:],optimize=True)        # i = [ini,nbeta]
    F[nalpha:nbf5,:,:] += np.einsum('i,imn->imn',n[nalpha:nbf5],J[nalpha:nbf5,:,:],optimize=True)  # i = [nalpha,nbf5]
          
    # C^J J
    F += np.einsum('ij,jmn->imn',cj12,J,optimize=True)                                                # i = [1,nbf5]
    F[ini:nbf5,:,:] -= np.einsum('ii,imn->imn',cj12[ini:nbf5,ini:nbf5],J[ini:nbf5,:,:],optimize=True) # quita i==j

    # -C^K K
    F -= np.einsum('ij,jmn->imn',ck12,K,optimize=True)                                                # i = [1,nbf5]
    F[ini:nbf5,:,:] += np.einsum('ii,imn->imn',ck12[ini:nbf5,ini:nbf5],K[ini:nbf5,:,:],optimize=True) # quita i==j
      
    return F

Definimos una función que reciba los orbitales moleculares y la matriz de Fock y calcule los multiplicadores de lagrange según
Generamos 
\begin{equation}
G_{\mu}^{(i)} = \sum_{\nu}^{N_{bf}} F_{\mu\nu}^{(i)} C_{\nu i}
\end{equation}

\begin{equation}
\lambda_{ij} = \sum_{\mu}^{N_{bf}} C_{\mu i} G_{\mu j}
\end{equation}

In [12]:
def computeLagrange(F,C):
    
    G = np.einsum('imn,ni->mi',F,C[:,0:nbf5],optimize=True)
            
    #Compute Lagrange multipliers
    elag = np.zeros((nbf,nbf))
    elag[0:noptorb,0:nbf5] = np.einsum('mi,mj->ij',C[:,0:noptorb],G,optimize=True)[0:noptorb,0:nbf5]
                
    return elag

Definimos una función para calcular la energía electrónica
\begin{equation}
E = \sum_{i=1}^{N_{\beta}} \left( \lambda_{ii} + n_i \sum_{\mu\nu} C_{\mu i}H_{\mu\nu}C_{\nu i} \right) + \sum_{i=N_{\beta}+1}^{N_{\alpha}} \left( \lambda_{ii} + n_i \sum_{\mu\nu} C_{\mu i}H_{\mu\nu}C_{\nu i} \right) + \sum_{i=N_{\alpha+1}}^{N_{bf5}} \left( \lambda_{ii} + n_i \sum_{\mu\nu} C_{\mu i}H_{\mu\nu}C_{\nu i} \right)
\end{equation}

In [13]:
def computeE_elec(H,C,n,elag):
    #EELECTRr
    E = 0

    E = E + np.einsum('ii',elag[:nbf5,:nbf5],optimize=True)
    E = E + np.einsum('i,mi,mn,ni',n[:nbf5],C[:,:nbf5],H,C[:,:nbf5],optimize=True)
          
    return E

Definimos una función que calcule la convergencia de los multiplicadores de lagrange

In [14]:
def computeLagrangeConvergency(elag):
    # Convergency
    
    sumdiff = np.sum(np.abs(elag-elag.T))
    maxdiff = np.max(np.abs(elag-elag.T))

    return sumdiff,maxdiff

Finalmente definimos una función que reciba coeficientes y haga lo siguiente
1. Calcule $J_{\mu\nu}^{(i)}$ y $K_{\mu\nu}^{(i)}$
2. Calcule la matriz $F_{\mu\nu}^{(i)}$
3. Calcule los multiplicadores de Lagrange $\lambda_{ij}$
4. Calcule la energía
5. Revise la convergencia de los multiplicadores de Lagrange

In [15]:
def ENERGY1r(C,n,cj12,ck12):
    
    J,K = computeJK(C)

    F = computeF(J,K,n,cj12,ck12)

    elag = computeLagrange(F,C)

    E = computeE_elec(H,C,n,elag)

    sumdiff,maxdiff = computeLagrangeConvergency(elag)        
        
    return E,elag,sumdiff,maxdiff

## Iterative Diagonalizator Hartree-Fock

Definimos valores iniciales de números de ocupación (n).

\begin{equation}
n_{i} = \left\{
  \begin{array}{lll}
  1     & \mathrm{si\ } i \in [1,N_{\beta}]  \\
  0.5   & \mathrm{si\ } i \in [N_{\beta},N_{\alpha}]  \\
  0     & \mathrm{si\ otro\ caso} 
  \end{array}
  \right.
\end{equation}

Los funcionales PNOF son del tipo
\begin{equation}
E = C^{J}_{ij} J_{MO} + C^{K}_{ij} K_{MO}
\end{equation}

Donde $C^{J}_{ij}$ y $C^{K}_{ij}$ son ciertos coeficientes que se determinan para cada funcional y cumplen con ciertas reglas.

Definiremos valores iniciales para un HF
\begin{eqnarray}
C^J_{ij} &=& 2n_in_j\\
C^K_{ij} &=& n_in_j
\end{eqnarray}

In [16]:
no1_ori = no1
no1 = nbeta

n = np.zeros((nbf5))
n[0:nbeta] = 1.0
n[nbeta:nalpha] = 0.5

cj12 = 2*np.einsum('i,j->ij',n,n)
ck12 = np.einsum('i,j->ij',n,n)

Definimos una función que reciba los multiplicadores de Lagrange y calcule Fmiug

En la primera iteración
\begin{equation}
Fmiug_{ij} = \frac{\lambda_{ij}+\lambda_{ji}}{2}
\end{equation}

En las demás
\begin{equation}
Fmiug_{ij} =  \theta(j-i) (\lambda_{ij}-\lambda_{ji}) + \theta(i-j) (\lambda_{ji}-\lambda_{ij})
\end{equation}

Donde $\theta(x)$ es la función de Heaviside

Además, se aplica una técnica de escalamiento a los elementos fuera de la diagonal, tal que
\begin{equation}
Fmiug_{ij} = 
  \begin{array}{lll}
  0.1Fmiug_{ij}     & \mathrm{si\ } 10^{N_{zeros}+9-k} < |Fmiug_{ij}| < 10^{N_{zeros}+10-k}  \\
  \end{array}
\end{equation}
con $k \in [0,N_{zeros}+9]$

In [17]:
def fmiug_scaling(fmiug0,elag,i_ext,nzeros):
    
    #scaling
    fmiug = np.zeros((nbf,nbf))
    if(i_ext == 0):
        fmiug[:noptorb,:noptorb] = ((elag[:noptorb,:noptorb] + elag[:noptorb,:noptorb].T) / 2)

    else:
        fmiug[:noptorb,:noptorb] = (elag[:noptorb,:noptorb] - elag[:noptorb,:noptorb].T)
        fmiug = np.tril(fmiug,-1) + np.tril(fmiug,-1).T
        for k in range(nzeros+9+1):
            fmiug[(abs(fmiug) > 10**(9-k)) & (abs(fmiug) < 10**(10-k))] *= 0.1
        np.fill_diagonal(fmiug[:noptorb,:noptorb],fmiug0[:noptorb])

    return fmiug

Creamos el SCF del HFIDr, el cual consiste en iteraciones externas e internas. En cada iteración interna se sigue el siguiente procedimiento:
1. Calcular $Fmiug$ a partir de los multiplicadores de lagrange $\varepsilon$, aplicando escalamiento de ser necesario
2. Diagonalizar $Fmiug$
3. Obtener nuevos coeficientes de orbitales moleculares
4. Calcular nueva matriz generalizada de Fock $F_{\mu\nu}^{(i)}$, así como nuevos multiplicadores de Lagrange $\varepsilon_{ij}$
5. Revisar convergencia

In [18]:
#HFIDr

print('{:^7} {:^7} {:^14} {:^14} {:^15} {:^14}'.format("Nitext","Nitint","Eelec","Etot","Ediff","maxdiff"))

E,elag,sumdiff,maxdiff = ENERGY1r(C,n,cj12,ck12)

fmiug0 = np.zeros((nbf))
ext = True
# iteraciones externas
for i_ext in range(maxitid):
    if i_ext==0:
        maxlp = 1
    else:
        maxlp = maxloop
            
    # iteraciones internas             
    for i_int in range(maxlp):
        E_old = E

        if(scaling):
            fmiug = fmiug_scaling(fmiug0,elag,i_ext,nzeros)
                   
        fmiug0, W = np.linalg.eigh(fmiug)
        C = np.matmul(C,W)
        E,elag,sumdiff,maxdiff = ENERGY1r(C,n,cj12,ck12)

        E_diff = E-E_old                    
        if(abs(E_diff)<thresheid):
            print('{:6d} {:6d} {:14.8f} {:14.8f} {:14.8f} {:14.8f}'.format(i_ext,i_int,E,E+E_nuc,E_diff,maxdiff))
            for i in range(nbf):
                fmiug0[i] = elag[i][i]
            ext = False
            break
    if(not ext):
        break
    print('{:6d} {:6d} {:14.8f} {:14.8f} {:14.8f} {:14.8f}'.format(i_ext,i_int,E,E+E_nuc,E_diff,maxdiff))

Nitext  Nitint      Eelec           Etot           Ediff         maxdiff    
     0      0   -71.66636066   -62.31174996     5.45167028     1.82522751
     1     29   -85.21544417   -75.86083347    -0.03469315     0.21879491
     2     29   -85.36234639   -76.00773569     0.00254138     0.09306746
     3     29   -85.37481556   -76.02020486    -0.00098278     0.05864438
     4     29   -85.38169791   -76.02708721    -0.00002715     0.01154111
     5     26   -85.38189755   -76.02728685    -0.00000099     0.00194318


Regresamos no1 a su estado original

In [19]:
no1 = no1_ori

## PNOF

A continuación generamos las $C_{ij}^{J}$ y $C_{ij}^{K}$ que definen a los funcionales PNOF, así como las derivadas para los números de ocupación. 

**Nota.** En $\frac{d C_{ij}^J}{d\gamma_k}$ y $\frac{d C_{ij}^K}{d\gamma_k}$ se aprovecha la simetría y solo se toma la mitad de la derivada, al calcular el gradiente se multiplicará por 2.

Para PNOF5

\begin{equation}
E^{PNOF5} = \sum_{g=1}^{N_{II}/2} \left[ \sum_{p \in \Omega_g} n_p (2H_{pp} + J_{pp}) + \sum_{q,p \in \Omega_g q \ne p} \underbrace{\Pi_{qp}^{g}}_{C^K_{pq}} K_{pq} \right] + \sum_{f \neq g}^{N_{II}/2} \sum_{p \in \Omega_f} \sum_{q \in \Omega_g} (\underbrace{2n_q n_p}_{C^J_{q p}} J_{pq}- \underbrace{n_q n_p}_{C^K_{q p}} K_{pq})
\end{equation}


\begin{equation}
C^J_{ij} = \left\{
  \begin{array}{lll}
  2 n_i n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
C^K_{ij} = \left\{
  \begin{array}{lll}
  n_i n_j   & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  -\Pi_{ij}       & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\Pi_{ij} = \left\{
  \begin{array}{lll}
   \sqrt{n_i n_j}      & \mathrm{si\ } i,j \in \Omega_{g} i=j\ \text{ó}\ i\ \text{y}\ j > \frac{N_{II}}{2}\\
  -\sqrt{n_i n_j}      & \mathrm{si\ } i,j \in \Omega_{g} i\ \text{ó}\ j \leq \frac{N_{II}}{2}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{dC^J_{ij}}{d\gamma_k} = \left\{
  \begin{array}{lll}
  2 \frac{d n_i}{d \gamma_k} n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{d C^K_{ij}}{d \gamma_k} = \left\{
  \begin{array}{lll}
  \frac{dn_i}{d\gamma_k} n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
 -\frac{d \sqrt{n_i n_j}}{d \gamma_k} = -\frac{1}{2\sqrt{n_i}} \frac{d n_i}{d \gamma_k} \sqrt{n_j} & \mathrm{si\ } i,j \in \Omega_{g} i=j\ \text{ó}\ i\ \text{y}\ j > \frac{N_{II}}{2}\\
  \frac{d \sqrt{n_i n_j}}{d \gamma_k} =  \frac{1}{2\sqrt{n_i}} \frac{d n_i}{d \gamma_k} \sqrt{n_j} & \mathrm{si\ } i,j \in \Omega_{g} i\ \text{ó}\ j \leq \frac{N_{II}}{2}\\  \end{array}
  \right.
\end{equation}

In [20]:
#CJCKD5
def CJCKD5(n):    
    
    cj12 = 2*np.einsum('i,j->ij',n,n)
    ck12 = np.einsum('i,j->ij',n,n)    
    
    for l in range(ndoc):            
        ldx = no1 + l
        # inicio y fin de los orbitales acoplados a los fuertemente ocupados
        ll = no1 + ndns+ncwo*(ndoc-l-1)
        ul = no1 + ndns+ncwo*(ndoc-l)

        cj12[ldx,ll:ul] = 0    
        cj12[ll:ul,ldx] = 0    
    
        cj12[ll:ul,ll:ul] = 0    
        
        ck12[ldx,ll:ul] = np.sqrt(n[ldx]*n[ll:ul])
        ck12[ll:ul,ldx] = np.sqrt(n[ldx]*n[ll:ul])

        ck12[ll:ul,ll:ul] = -np.outer(np.sqrt(n[ll:ul]),np.sqrt(n[ll:ul]))

    return cj12,ck12        
        
def der_CJCKD5(n,gamma,dn_dgamma):

    Dcj12r = 2*np.einsum('ik,j->ijk',dn_dgamma,n)    
    Dck12r = np.einsum('ik,j->ijk',dn_dgamma,n)    

    for l in range(ndoc):            
        ldx = no1 + l

        # inicio y fin de los orbitales acoplados a los fuertemente ocupados
        ll = no1 + ndns+ncwo*(ndoc-l-1)
        ul = no1 + ndns+ncwo*(ndoc-l)

        Dcj12r[ldx,ll:ul,:nv] = 0
        Dcj12r[ll:ul,ldx,:nv] = 0

        Dcj12r[ll:ul,ll:ul,:nv] = 0   
        
        a = n[ldx] 
        if(a<10**-12):
            a = 10**-12
        b = n[ll:ul]
        if(b<10**-12):
            b = 10**-12
        
        Dck12r[ldx,ll:ul,:nv] = 1/2 * np.sqrt(1/a)*dn_dgamma[ldx,:nv]*np.sqrt(n[ll:ul])
#        Dck12r[ll:ul,ldx,:nv] = 1/2 * np.sqrt(1/b)*dn_dgamma[ll:ul,:nv]*np.sqrt(n[ldx])
        Dck12r[ll:ul,ldx,:nv] = 1/2 * np.sqrt(1/b)*dn_dgamma[ll:ul,:nv]*np.sqrt(n[ldx])
        
        for k in range(nv):
            Dck12r[ll:ul,ll:ul,k] = -1/2 * dn_dgamma[ll:ul,k]   
                        
    return Dcj12r,Dck12r

Para PNOF7

\begin{equation}
E^{PNOF7} = \sum_{g=1}^{N_{II}/2} \left[ \sum_{p \in \Omega_g} n_p (2H_{pp} + J_{pp}) + \sum_{q,p \in \Omega_g q \ne p} \underbrace{\Pi_{qp}^{g}}_{C^K_{pq}} K_{pq} \right] + \sum_{f \neq g}^{N_{II}/2} \sum_{p \in \Omega_f} \sum_{q \in \Omega_g} (\underbrace{2n_q n_p}_{C^J_{q p}} J_{pq}- \underbrace{n_q n_p + \Phi_q \Phi_p}_{C^K_{q p}} K_{pq})
\end{equation}

\begin{equation}
\Phi_i = \sqrt{n_i(1-n_i)}
\end{equation}


\begin{equation}
C^J_{ij} = \left\{
  \begin{array}{lll}
  2 n_i n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
C^K_{ij} = \left\{
  \begin{array}{lll}
  n_i n_j + \Phi_i \Phi_j   & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  -\Pi_{ij}       & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\Pi_{ij} = \left\{
  \begin{array}{lll}
   \sqrt{n_i n_j}      & \mathrm{si\ } i,j \in \Omega_{g} i=j\ \text{ó}\ i\ \text{y}\ j > \frac{N_{II}}{2}\\
  -\sqrt{n_i n_j}      & \mathrm{si\ } i,j \in \Omega_{g} i\ \text{ó}\ j \leq \frac{N_{II}}{2}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{d \Phi_i}{d\gamma_k} = \left\{
  \begin{array}{lll}
  \frac{d \sqrt{n_i(1-n_i)}}{d \gamma_k} = \frac{1}{2 \sqrt{n_i(1-n_i)}} (1 - 2n_i) \frac{d n_i}{d \gamma_i} \delta_{ik} & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{dC^J_{ij}}{d\gamma_k} = \left\{
  \begin{array}{lll}
  2 \frac{d n_i}{d \gamma_k} n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{d C^K_{ij}}{d \gamma_k} = \left\{
  \begin{array}{lll}
  \frac{dn_i}{d\gamma_k} n_j + \frac{d\Phi_i}{d\gamma_k} \Phi_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
 -\frac{d \sqrt{n_i n_j}}{d \gamma_k} = -\frac{1}{2\sqrt{n_i}} \frac{d n_i}{d \gamma_k} \sqrt{n_j} & \mathrm{si\ } i,j \in \Omega_{g} i=j\ \text{ó}\ i\ \text{y}\ j > \frac{N_{II}}{2}\\
  \frac{d \sqrt{n_i n_j}}{d \gamma_k} =  \frac{1}{2\sqrt{n_i}} \frac{d n_i}{d \gamma_k} \sqrt{n_j} & \mathrm{si\ } i,j \in \Omega_{g} i\ \text{ó}\ j \leq \frac{N_{II}}{2}\\  \end{array}
  \right.
\end{equation}

In [21]:
#CJCKD7
def CJCKD7(n):    
    
    fi = n*(1-n)
    fi[fi<=0] = 0
    fi = np.sqrt(fi)      
    
    cj12 = 2*np.einsum('i,j->ij',n,n)
    ck12 = np.einsum('i,j->ij',n,n)    
    ck12 += np.einsum('i,j->ij',fi,fi)
    
    for l in range(ndoc):            
        ldx = no1 + l
        # inicio y fin de los orbitales acoplados a los fuertemente ocupados
        ll = no1 + ndns+ncwo*(ndoc-l-1)
        ul = no1 + ndns+ncwo*(ndoc-l)

        cj12[ldx,ll:ul] = 0    
        cj12[ll:ul,ldx] = 0    
    
        cj12[ll:ul,ll:ul] = 0    
        
        ck12[ldx,ll:ul] = np.sqrt(n[ldx]*n[ll:ul])
        ck12[ll:ul,ldx] = np.sqrt(n[ldx]*n[ll:ul])

        ck12[ll:ul,ll:ul] = -np.outer(np.sqrt(n[ll:ul]),np.sqrt(n[ll:ul]))

    return cj12,ck12        
        
def der_CJCKD7(n,dn_dgamma):
    
    fi = n*(1-n)
    fi[fi<=0] = 0
    fi = np.sqrt(fi)      
            
    dfi_dgamma = np.zeros((nbf5,nv))
    for i in range(no1,nbf5):
        a = fi[i]
        if(a < 10**-12):
            a = 10**-12
        for k in range(nv):
            dfi_dgamma[i,k] = 1/(2*a)*(1-2*n[i])*dn_dgamma[i][k]
    
    Dcj12r = 2*np.einsum('ik,j->ijk',dn_dgamma,n)    
    Dck12r = np.einsum('ik,j->ijk',dn_dgamma,n)    
    Dck12r += np.einsum('ik,j->ijk',dfi_dgamma,fi)    

    for l in range(ndoc):            
        ldx = no1 + l

        # inicio y fin de los orbitales acoplados a los fuertemente ocupados
        ll = no1 + ndns+ncwo*(ndoc-l-1)
        ul = no1 + ndns+ncwo*(ndoc-l)

        Dcj12r[ldx,ll:ul,:nv] = 0
        Dcj12r[ll:ul,ldx,:nv] = 0

        Dcj12r[ll:ul,ll:ul,:nv] = 0   

        a = n[ldx] 
        if(a<10**-12):
            a = 10**-12
        b = n[ll:ul]
        if(b<10**-12):
            b = 10**-12        
        
        Dck12r[ldx,ll:ul,:nv] = 1/2 * np.sqrt(1/a)*dn_dgamma[ldx,:nv]*np.sqrt(n[ll:ul])
        Dck12r[ll:ul,ldx,:nv] = 1/2 * np.sqrt(1/b)*dn_dgamma[ll:ul,:nv]*np.sqrt(n[ldx])
        
        for k in range(nv):
            Dck12r[ll:ul,ll:ul,k] = - 1/2 * dn_dgamma[ll:ul,k]
                        
    return Dcj12r,Dck12r

Creamos un seleccionador de PNOF

In [22]:
def PNOFi_selector(PNOFi,n):
    if(PNOFi==5):
        cj12,ck12 = CJCKD5(n)
    if(PNOFi==7):
        cj12,ck12 = CJCKD7(n)
        
    return cj12,ck12

def der_PNOFi_selector(PNOFi,n,dn_dgamma):
    if(PNOFi==5):
        Dcj12r,Dck12r = der_CJCKD5(n,dn_dgamma)
    if(PNOFi==7):
        Dcj12r,Dck12r = der_CJCKD7(n,dn_dgamma)
        
    return Dcj12r,Dck12r

## Optimización de Ocupaciones

Declaramos el número de variables en la optimización de ocupaciones
\begin{equation}
N_v = N_{cwo}N_{doc}
\end{equation}

In [23]:
nv = ncwo*ndoc

Definimos una función para calcular $J_{MO}$ y $K_{MO}$
\begin{eqnarray}
{J_{MO}}_{ij} &=& \sum_{\mu\nu} D^{(j)}_{\mu\nu} J^{(i)}_{\mu\nu}\\
{K_{MO}}_{ij} &=& \sum_{\mu\sigma} D^{(j)}_{\mu\sigma} K^{(i)}_{\mu\sigma}\\
{H_{core}}_{i} &=& \sum_{\mu\nu} D^{(i)}_{\mu\nu} H_{\mu\nu}
\end{eqnarray}


In [24]:
def computeJKH_core_MO(C,H):

    #denmatj    
    D = np.einsum('mi,ni->imn', C[:,0:nbf5], C[:,0:nbf5], optimize=True)

    #QJMATm
    J = np.einsum('isl,mnsl->imn', D, I, optimize=True)    
    J_MO = np.einsum('jmn,imn->ij', D, J, optimize=True)    
        
    #QKMATm        
    K = np.einsum('inl,mnsl->ims', D, I, optimize=True)    
    K_MO = np.einsum('jms,ims->ij', D, K, optimize=True)        

    #QHMATm
    H_core = np.einsum('imn,mn->i', D, H, optimize=True)

    return J_MO,K_MO,H_core

Definimos una función que calcule los números de ocupación y sus derivadas respecto a $\gamma$

RO:
\begin{equation}
n_i = \left\{
  \begin{array}{lll}
   1      & \mathrm{si\ } i \in [1,N_{o1}]\\
   \frac{1}{2} (1 + cos^2(\gamma_{i-N_{doc}}))     & \mathrm{si\ } i \in (N_{o1},N_{\beta}]\\
   \frac{1}{2} sin^2(\gamma_{N_{doc}-i-1})     & \mathrm{si\ } i \in (N_{\alpha},N_{bf5}]\\
  \end{array}
  \right.
\end{equation}


DRO:
\begin{equation}
\frac{d n_i}{d \gamma_k} = \left\{
  \begin{array}{lll}
   0      & \mathrm{si\ } i \in [1,N_{o1}]\\
   -\frac{1}{2} sin(2\gamma_{i-N_{doc}}) \delta_{ik}     & \mathrm{si\ } i \in (N_{o1},N_{\beta}]\\
   \frac{1}{2} sin(2\gamma_{N_{doc}-i-1})     & \mathrm{si\ } i \in (N_{\alpha},N_{bf5}]\\
  \end{array}
  \right.
\end{equation}

In [25]:
def ocupacion(gamma):
        
    n = np.zeros((nbf5))
    dni_dgammai = np.zeros((nbf5))

    n[0:no1] = 1                                              # [1,no1]

    n[no1:no1+ndoc] = 1/2 * (1 + np.cos(gamma[:ndoc])**2)     # (no1,no1+ndoc]
    dni_dgammai[no1:no1+ndoc] = - 1/2 * np.sin(2*gamma[:ndoc])
       
    if(ncwo==1):
        dn_dgamma = np.zeros((nbf5,nv))    

        for i in range(ndoc):
            dn_dgamma[no1+i][i] = dni_dgammai[no1+i]
            #cwo
            icf = nalpha+ndoc - i - 1
            n[icf] = 1/2*np.sin(gamma[i])**2
            dni_dgammai[icf]  = 1/2*np.sin(2*gamma[i])
            dn_dgamma[icf][i] = dni_dgammai[icf]

    return n,dn_dgamma

**Generamos gamma**

\begin{equation}
\gamma_{k} = \left\{
  \begin{array}{lll}
  cos^{-1}(\sqrt{2\times0.999-1})      & \mathrm{si\ } k \in N_{doc}  \\
  sin^{-1}(\sqrt{\frac{1}{N_{cwo}-j+1}})     & \mathrm{si\ } k = N_{doc} + (i-1)(N_{cwo}-1)+j;i\in N_{doc};j\in N_{cwo}-1
  \end{array}
  \right.
\end{equation}

Esto se hace considerando la relación
\begin{equation}
n_g = \frac{1}{2} (1 + cos^2 \gamma_g) ; g= 1,2,\cdots,N_{II}/2
\end{equation}


In [26]:
gamma = np.zeros((nbf5))
for i in range(ndoc):
    gamma[i] = np.arccos(np.sqrt(2.0*0.999-1.0))
    for j in range(ncwo-1):
        ig = ndoc+(i)*(ncwo-1)+j
        gamma[ig] = np.arcsin(np.sqrt(1.0/(ncwo-j)))        

**Definimos una función que recibe ($\gamma$) y calcula la energía**

\begin{eqnarray}
E &=& \sum^{N_{o1}}_{i=1} \left[ n_i (2{H_{core}}_i + {J_{MO}}_{ii}) + \sum^{N_{bf5}}_{j \neq i} C^J_{ij} {J_{MO}}_{ji} - C^K_{ij} {K_{MO}}_{ji} \right]\\
&+& \sum^{N_{doc}}_{i=N_{o1}+1} \left[ n_i (2{H_{core}}_i + {J_{MO}}_{ii}) + \sum^{N_{bf5}}_{j \neq i} C^J_{ij} {J_{MO}}_{ji} - C^K_{ij} {K_{MO}}_{ji} \right]\\
&+& \sum^{(N_{doc}+1)N_{cwo}}_{i=N_{doc}+1} \left[ n_i (2{H_{core}}_i + {J_{MO}}_{ii}) + \sum^{N_{bf5}}_{j \neq i} C^J_{ij} {J_{MO}}_{ji} - C^K_{ij} {K_{MO}}_{ji} \right]\\
\end{eqnarray}

In [27]:
#calce
def calce(gamma,J_MO,K_MO,H_core,PNOFi):
    
    n,dn_dgamma = ocupacion(gamma)
    cj12,ck12 = PNOFi_selector(PNOFi,n)
    
    E = 0

    # 2H + J
    E = E + np.einsum('i,i',n[:nbeta],2*H_core[:nbeta]+np.diagonal(J_MO)[:nbeta]) # [0,Nbeta]
    E = E + np.einsum('i,i',n[nbeta:nalpha],2*H_core[nbeta:nalpha])               # (Nbeta,Nalpha]
    E = E + np.einsum('i,i',n[nalpha:nbf5],2*H_core[nalpha:nbf5]+np.diagonal(J_MO)[nalpha:nbf5]) # (Nalpha,Nbf5)

    #C^J JMO
    E = E + np.einsum('ij,ji',cj12,J_MO) # sum_ij
    E = E - np.einsum('ii,ii',cj12,J_MO) # Quita i=j

    #C^K KMO     
    E = E - np.einsum('ij,ji',ck12,K_MO) # sum_ij
    E = E + np.einsum('ii,ii',ck12,K_MO) # Quita i=j
            
    return E

Definimos una función que recibe ($\gamma$) y calcula el gradiente

\begin{equation}
\frac{dE}{d\gamma_{k}} = \sum_{i=N_{o1}+1}^{N_{\beta}} \left[ \frac{dn_i}{d \gamma_{k}} (2{H_{core}}_{i} + {J_{MO}}_{ii}) + \sum_{j \neq i}^{N_{bf5}} 2 \frac{d C^J_{ij}}{d \gamma_{k}}{J_{MO}}_{ji} - 2 \frac{d C^K_{ij}}{d \gamma_{k}}{K_{MO}}_{ji} \right] + \sum_{i=N_{\alpha}+1}^{N_{bf5}} \left[ \frac{dn_i}{d \gamma_{k}} (2{H_{core}}_{i} + {J_{MO}}_{ii}) + \sum_{j \neq i}^{N_{bf5}} 2 \frac{d C^J_{ij}}{d \gamma_{k}}{J_{MO}}_{ji} - 2 \frac{d C^K_{ij}}{d \gamma_{k}}{K_{MO}}_{ji} \right]
\end{equation}

In [28]:
#calcg
def calcg(gamma,J_MO,K_MO,H_core,PNOFi):
    
    grad = np.zeros((nv))

    n,dn_dgamma = ocupacion(gamma)    
    Dcj12r,Dck12r = der_PNOFi_selector(PNOFi,n,dn_dgamma)

    # dn_dgamma (2H+J)
    grad += np.einsum('ik,i->k',dn_dgamma[no1:nbeta,:nv],2*H_core[no1:nbeta]+np.diagonal(J_MO)[no1:nbeta],optimize=True) # [0,Nbeta]
    grad += np.einsum('ik,i->k',dn_dgamma[nalpha:nbf5,:nv],2*H_core[nalpha:nbf5]+np.diagonal(J_MO)[nalpha:nbf5],optimize=True) # [Nalpha,Nbf5]

    # 2 dCJ_dgamma J_MO
    grad += 2*np.einsum('ijk,ji->k',Dcj12r[no1:nbeta,:nbf5,:nv],J_MO[:nbf5,no1:nbeta],optimize=True)
    grad -= 2*np.einsum('iik,ii->k',Dcj12r[no1:nbeta,no1:nbeta,:nv],J_MO[no1:nbeta,no1:nbeta],optimize=True)

    grad += 2*np.einsum('ijk,ji->k',Dcj12r[nalpha:nbf5,:nbf5,:nv],J_MO[:nbf5,nalpha:nbf5],optimize=True)
    grad -= 2*np.einsum('iik,ii->k',Dcj12r[nalpha:nbf5,nalpha:nbf5,:nv],J_MO[nalpha:nbf5,nalpha:nbf5],optimize=True)
      
    # -2 dCK_dgamma K_MO    
    grad -= 2*np.einsum('ijk,ji->k',Dck12r[no1:nbeta,:nbf5,:nv],K_MO[:nbf5,no1:nbeta],optimize=True)
    grad += 2*np.einsum('iik,ii->k',Dck12r[no1:nbeta,no1:nbeta,:nv],K_MO[no1:nbeta,no1:nbeta],optimize=True)

    grad -= 2*np.einsum('ijk,ji->k',Dck12r[nalpha:nbf5,:nbf5,:nv],K_MO[:nbf5,nalpha:nbf5],optimize=True)
    grad += 2*np.einsum('iik,ii->k',Dck12r[nalpha:nbf5,nalpha:nbf5,:nv],K_MO[nalpha:nbf5,nalpha:nbf5],optimize=True)   

    return grad

**Definimos una función que optimiza los numeros de ocupación utilizando la energía como función objetivo.**

In [29]:
def occoptr(gamma,firstcall,convgdelag,elag,C,H):
        
    J_MO,K_MO,H_core = computeJKH_core_MO(C,H)
    
    if (not convgdelag):
        if(gradient=="analytical"):
            res = minimize(calce, gamma[:nv], args=(J_MO,K_MO,H_core,PNOFi), jac=calcg, method='CG')
        elif(gradient=="numerical"):
            res = minimize(calce, gamma[:nv], args=(J_MO,K_MO,H_core,PNOFi),  method='CG')
        gamma = res.x
    n,DR = ocupacion(gamma)
    cj12,ck12 = PNOFi_selector(PNOFi,n)
        
    if (firstcall):
        elag_diag = np.zeros((nbf))
               
        # RO (H_core + J)
        elag_diag[:nbeta] = np.einsum('i,i->i',n[:nbeta],H_core[:nbeta]+np.diagonal(J_MO)[:nbeta])        
        elag_diag[nbeta:nalpha] = np.einsum('i,i->i',n[nbeta:nalpha],H_core[nbeta:nalpha])        
        elag_diag[nalpha:nbf5] = np.einsum('i,i->i',n[nalpha:nbf5],H_core[nalpha:nbf5]+np.diagonal(J_MO)[nalpha:nbf5])        

        # CJ12 J_MO
        elag_diag[:nbf5] += np.einsum('ij,ji->i',cj12,J_MO)
        elag_diag[:nbf5] -= np.einsum('ii,ii->i',cj12,J_MO)

        # CK12 K_MO
        elag_diag[:nbf5] -= np.einsum('ij,ji->i',ck12,K_MO)
        elag_diag[:nbf5] += np.einsum('ii,ii->i',ck12,K_MO)
        
#        for i in range(nbf):
#            elag[i][i] = elag_diag[i]
    
    return gamma,elag,n,cj12,ck12

Hacemos una primera optimización de los números de ocupación

In [30]:
gamma,elag,n,cj12,ck12 = occoptr(gamma,True,False,elag,C,H)

## Optimización Orbital y (de ocupaciones)

Definimos una función para DIIS

In [31]:
def fmiug_diis(fk,fmiug,idiis,bdiis,maxdiff):
    
    if(maxdiff<thdiis):

        restart_diis = False
        fk[idiis,0:noptorb,0:noptorb] = fmiug[0:noptorb,0:noptorb]
        for m in range(idiis+1):
            bdiis[m][idiis] = 0
            for i in range(noptorb):
                for j in range(i):
                    bdiis[m][idiis] = bdiis[m][idiis] + fk[m][i][j]*fk[idiis][j][i]
            bdiis[idiis][m] = bdiis[m][idiis]
            bdiis[m][idiis+1] = -1
            bdiis[idiis+1][m] = -1
        bdiis[idiis+1][idiis+1] = 0  
                        
        if(idiis>=ndiis):
            
            cdiis = np.zeros((idiis+2))
            cdiis[0:idiis+1] = 0
            cdiis[idiis+1] = -1
            x = np.linalg.solve(bdiis[0:idiis+2,0:idiis+2],cdiis[0:idiis+2])
            
            for i in range(noptorb):
                for j in range(i):
                    fmiug[i][j] = 0
                    for k in range(idiis+1):
                        fmiug[i][j] = fmiug[i][j] + x[k]*fk[k][i][j]
                    fmiug[j][i] = fmiug[i][j]
                                        
            restart_diis=True
        idiis = idiis + 1    
        if(restart_diis):
            idiis = 0
            
    return fk,fmiug,idiis,bdiis

Creamos una función para la optimización orbital

In [32]:
def orboptr(C,n,cj12,ck12,E_old,sumdiff_old,i_ext,itlim,nzeros,fmiug0):

    convgdelag = False

    E,elag,sumdiff,maxdiff = ENERGY1r(C,n,cj12,ck12)

    E_diff = E-E_old                    
    P_CONV = abs(E_diff)
    E_old = E
    
    if(maxdiff<threshl and abs(E_diff)<threshe):
        convgdelag = True
        print('{:6d} {:6d} {:14.8f} {:14.8f} {:14.8f} {:14.8f}'.format(i_ext,0,E,E+E_nuc,E_diff,maxdiff))        
        return convgdelag,E_old,sumdiff_old,itlim,nzeros,fmiug0,C    
                
    if (scaling and i_ext>1 and i_ext > itlim and sumdiff > sumdiff_old):
        nzeros = nzeros + 1
        itlim = (i_ext + 1) + itziter
        if (nzeros>nzerosm):
            nzeros = nzerosr
    sumdiff_old = sumdiff
     
    if i_ext==0:
        maxlp = 1
    else:
        maxlp = 30
    
    fmiug = np.zeros((noptorb,noptorb))
    fk = np.zeros((30,noptorb,noptorb))
    bdiis = np.zeros((31,31))
    cdiis = np.zeros((31))        
    iloop = 0
    idiis = 0
    
    for i_int in range(maxlp):
        iloop = iloop + 1
        E_old2 = E

        #scaling
        fmiug = fmiug_scaling(fmiug0,elag,i_ext,nzeros)
        if(diis and maxdiff < thdiis):
            fk,fmiug,idiis,bdiis = fmiug_diis(fk,fmiug,idiis,bdiis,maxdiff)   
          
        eigval, eigvec = np.linalg.eigh(fmiug)
        fmiug0 = eigval
                
        C = np.matmul(C,eigvec)

        E,elag,sumdiff,maxdiff = ENERGY1r(C,n,cj12,ck12)

        E_diff2 = E-E_old2                    
                       
        if(abs(E_diff2)<threshec or i_int==maxlp-1):
            E_diff = E-E_old
            E_old = E
            print('{:6d} {:6d} {:14.8f} {:14.8f} {:14.8f} {:14.8f}'.format(i_ext+1,i_int,E,E+E_nuc,E_diff,maxdiff))
            break    
    
    return convgdelag,E_old,sumdiff_old,itlim,nzeros,fmiug0,C

## SCF de PNOF

In [33]:
iloop = 0
itlim = 0
E_old = E
sumdiff_old = 0

print('{:^7} {:^7} {:^14} {:^14} {:^14} {:^14}'.format("Nitext","Nitint","Eelec","Etot","Ediff","maxdiff"))

for i_ext in range(maxit):
    
    #orboptr
    convgdelag,E_old,sumdiff_old,itlim,nzeros,fmiug0,C = orboptr(C,n,cj12,ck12,E_old,sumdiff_old,i_ext,itlim,nzeros,fmiug0)     
    
    #occopt
    gamma,elag,n,cj12,ck12 = occoptr(gamma,False,convgdelag,elag,C,H)
        
    if(convgdelag):
        break

Nitext  Nitint      Eelec           Etot          Ediff         maxdiff    
     1      0   -85.38391402   -76.02930332    -0.00024450     0.00746295
     2     29   -85.28435185   -75.92974114     0.09978945     0.14618216
     3     29   -85.40094959   -76.04633888    -0.11413285     0.02031975
     4     29   -85.43140118   -76.07679047    -0.01773303     0.01376386
     5     29   -85.44537092   -76.09076022    -0.00865994     0.02460318
     6     29   -85.44822328   -76.09361258    -0.00205230     0.01926152
     7     29   -85.45018423   -76.09557352    -0.00161697     0.01923268
     8     29   -85.45071576   -76.09610505    -0.00045203     0.01554229
     9     29   -85.45147765   -76.09686695    -0.00074975     0.01039666
    10     29   -85.45041192   -76.09580121     0.00107183     0.01985975
    11     29   -85.45082198   -76.09621128    -0.00040648     0.01897242
    12     29   -85.45162644   -76.09701573    -0.00080355     0.01363702
    13     29   -85.45137640   -76.0

In [34]:
for i in range(C.shape[0]):
    print("------------------")
    for j in range(C.shape[0]):
        print(i,j,C[j][i])

------------------
0 0 1.0010244507963426
0 1 -0.006444671963155734
0 2 -0.01009143023669349
0 3 2.3249135915661782e-05
0 4 4.152905337652729e-07
0 5 -0.005115655568748825
0 6 1.8094931736436994e-05
0 7 1.0158412292400521e-06
0 8 -0.003177085070804919
0 9 -0.0013354697373476664
0 10 5.571705447235678e-07
0 11 -1.8780515171555962e-06
0 12 -0.0011197504966865732
0 13 -1.487514219527394e-06
0 14 -0.000989686290656862
0 15 -0.001762272659515192
0 16 0.0003375687103506042
0 17 9.187390120345611e-07
0 18 0.000833880624746718
0 19 -0.0007735307861145128
0 20 -0.0017593807623172448
0 21 0.00033908620442863
0 22 1.542782350345591e-06
0 23 -0.0008325273785094288
0 24 -0.0007722315347200758
------------------
1 0 -0.0026960666099127392
1 1 -0.14447973339271308
1 2 0.011356976356234783
1 3 -4.539830345793315e-05
1 4 0.3513934849198742
1 5 0.3186852206494891
1 6 -4.680752555302939e-05
1 7 0.14459914354509287
1 8 0.17099915459531398
1 9 0.004035556367715383
1 10 -1.2077713083514816e-05
1 11 4.660633

# Proyección

In [35]:
# Set computation options
old_basis = wfn.basisset()

psi4.set_options({'basis': 'cc-pVDZ',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})

In [36]:
from scipy.linalg import orth
from numpy.linalg import matrix_power
import scipy

#aux = psi4.core.BasisSet.build(mol, "ORBITAL", "")
#old_basis = wfn.basisset()

wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))
mints = psi4.core.MintsHelper(wfn.basisset())

M = np.asarray(mints.ao_overlap(wfn.basisset(),old_basis))

Sinv = mints.ao_overlap()
Sm12 = mints.ao_overlap()
Sp12 = mints.ao_overlap()
Sinv.power(-1.0, 1.e-14)
Sm12.power(-0.5, 1.e-14)
Sp12.power(0.5, 1.e-14)

Cnew = np.matmul(np.matmul(Sinv,M),C)

#Cnew = np.matmul(Sp12,Cnew)

#Cnew = orth(Cnew)

#Cnew = np.matmul(Sm12,Cnew)

#Q = np.matmul(Cnew.T,Cnew)
#Q = psi4.core.Matrix.from_array(Q)
#Q.power(-0.5,1.e-16)

#Cneww = scipy.linalg.orth(Cneww)
C = np.zeros((Cnew.shape[0],Cnew.shape[0]))
C[:,:Cnew.shape[1]] = Cnew

for i in range(Cnew.shape[1]):
    print("------------------")
    for j in range(Cnew.shape[0]):
        print(i,j,Cnew[j][i])#,Cneww[j][i])        

------------------
0 0 1.0010244507963413
0 1 -0.006444671963155109
0 2 -0.010091430236689047
0 3 2.3249135915660288e-05
0 4 4.1529053395397883e-07
0 5 -0.005115655568749004
0 6 1.8094931736245673e-05
0 7 1.0158412310901534e-06
0 8 -0.003177085070806452
0 9 -0.001335469737349229
0 10 5.571705447556651e-07
0 11 -1.878051516808369e-06
0 12 -0.0011197504966866394
0 13 -1.4875142207080031e-06
0 14 -0.0009896862906574903
0 15 -0.0017622726595173473
0 16 0.00033756871034885076
0 17 9.187390121175949e-07
0 18 0.0008338806247477989
0 19 -0.000773530786115576
0 20 -0.0017593807623183801
0 21 0.00033908620442969854
0 22 1.542782350555678e-06
0 23 -0.0008325273785095757
0 24 -0.0007722315347199454
------------------
1 0 -0.002696066609913132
1 1 -0.14447973339271372
1 2 0.011356976356240498
1 3 -4.539830345782994e-05
1 4 0.3513934849198745
1 5 0.3186852206494882
1 6 -4.6807525553028066e-05
1 7 0.14459914354509415
1 8 0.17099915459531267
1 9 0.004035556367714882
1 10 -1.2077713083505946e-05
1 11 4

Construimos la función de onda y **evaluamos matrices e integrales en orbital atómico**, $S$, $T$, $V$, $H$, $(\mu\nu|\sigma\lambda)$

In [37]:
# Wavefunction
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))

# Integrador
mints = psi4.core.MintsHelper(wfn.basisset())

# Overlap, Kinetics, Potential
S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V

# Integrales de Repulsión Electrónica, ERIs (mu nu | sigma lambda)
I = np.asarray(mints.ao_eri())

# Energía Nuclear
E_nuc = mol.nuclear_repulsion_energy()

# Variables

In [38]:
natoms = mol.natom()
nbf = S.shape[0]
nalpha = wfn.nalpha()
nbeta = wfn.nbeta()
ne = nalpha + nbeta
mul = mol.multiplicity()
no1 = 0 #Number of inactive doubly occupied orbitals | Se puede variar
for i in range(natoms):
    Z = mol.Z(i)
    if ( 1<=Z and Z<=  2):
        no1 += 0           # H-He
    elif ( 3<=Z and Z<= 10):
        no1 +=  1          # Li-Ne
    elif (11<=Z and Z<= 18):
        no1 +=  5          # Na-Ar
    elif(19<=Z and Z<= 36):
        no1 +=  9          # K-Kr
    elif(37<=Z and Z<= 49):
        no1 += 18          # Rb-In
    elif(50<=Z and Z<= 54):
        no1 += 23          # Sn-Xe
    elif(55<=Z and Z<= 71):
        no1 += 27          # Cs-Lu
    elif(72<=Z and Z<= 81):
        no1 += 30          # Hf-Tl
    elif(82<=Z and Z<= 86):
        no1 += 39          # Pb-Rn
    elif(87<=Z and Z<=109):
        no1 += 43          # Fr-Mt
    
ndoc = nbeta   -   no1
nsoc = nalpha  -   nbeta
ndns = ndoc    +   nsoc
nvir = nbf     -   nalpha

ncwo = 1
if(ne==2):
    ncwo= -1
if(ndns!=0):
    if(ndoc>0):
        if(ncwo!=1):
            if(ncwo==-1 or ncwo > nvir/ndoc):
                ncwo = int(nvir/ndoc)
    else:
        ncwo = 0

nac = ndoc * (1 + ncwo)
nbf5 = no1 + nac + nsoc   #JFHLY warning: nbf must be >nbf5
no0 = nbf - nbf5

noptorb = nbf     

closed = (nbeta == (ne+mul-1)/2 and nalpha == (ne-mul+1)/2)

Establecemos algunos parámetros

In [39]:
maxit = 1000  # Número máximo de iteraciones de Occ-SCF
no1 = no1     # Número de orbitales inactivos con ocupación 1
thresheid = 10**-8 # Convergencia de la energía total
maxitid = 30  # Número máximo de iteraciones externas
ipnof = 5     # PNOFi a calcular
threshl = 10**-4   # Convergencia de los multiplicadores de Lagrange
threshe = 10**-6   # Convergencia de la energía
threshec = 10**-10 # Convergencia  de la energía en optimización orbital
threshen = 10**-10 # Convergencia  de la energía en optimización de ocupaciones
scaling = True     # Scaling for f
nzeros = 0
nzerosm = 4
nzerosr = 2
itziter = 10        # Iteraciones para scaling constante
diis = True         # DIIS en optimización orbital
thdiis = 10**-3     # Para iniciar DIIS
ndiis = 5           # Número de ciclos para interpolar matriz de Fock generalizada en DIIS
perdiis = True      # Aplica DIIS cada NDIIS (True) o después de NDIIS (False) 
ncwo = ncwo         # Número de orbitales débilmente ocupados acoplados a cada orbital fueremtente ocupado 
noptorb = noptorb   # Número de orbitales a optimizar Nbf5 <= Noptorb <= Nbf
scaling = True

Obtenemos un Guess acorde a la ecuación

\begin{equation}
HC = SC\varepsilon
\end{equation}

In [40]:
import numpy as np
from numpy.linalg import lstsq
from scipy.linalg import orth

E_i,Ctmp = eigh(H, S)
C[:,Cnew.shape[1]:] = Ctmp[:,Cnew.shape[1]:]

Cp = np.matmul(Sp12,C)

def find_orth(O,vec):
    A = np.hstack((O, vec.reshape((O.shape[0],1))))
    b = np.zeros(O.shape[1] + 1)
    b[-1] = 1
    return lstsq(A.T, b)[0]

Cpnew = Cp[:,0]
Cpnew = Cpnew.reshape((Cpnew.shape[0],1))

for i in range(1,C.shape[1]):
    res = find_orth(Cpnew,Cp[:,i])
    res = res/np.linalg.norm(res)

    Cpnew = np.hstack((Cpnew,res.reshape((Cpnew.shape[0],1))))

C = np.matmul(Sm12,Cpnew)

/home/jfhlewyee/anaconda3/envs/pydonof/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [41]:
for i in range(C.shape[0]):
    print("------------------")
    for j in range(C.shape[0]):
        print(i,j,C[j][i])

------------------
0 0 1.0010244507963408
0 1 -0.0064446719631541536
0 2 -0.010091430236688433
0 3 2.3249135915730328e-05
0 4 4.1529053396997854e-07
0 5 -0.005115655568748781
0 6 1.8094931736130358e-05
0 7 1.0158412308260159e-06
0 8 -0.0031770850708061904
0 9 -0.0013354697373497424
0 10 5.571705447517103e-07
0 11 -1.8780515167539639e-06
0 12 -0.0011197504966867055
0 13 -1.4875142202929526e-06
0 14 -0.0009896862906579363
0 15 -0.0017622726595171666
0 16 0.0003375687103489675
0 17 9.18739012281191e-07
0 18 0.0008338806247476347
0 19 -0.0007735307861157602
0 20 -0.0017593807623191792
0 21 0.0003390862044295434
0 22 1.542782350854956e-06
0 23 -0.0008325273785096523
0 24 -0.000772231534720301
------------------
1 0 -0.002696066609912963
1 1 -0.14447973339271086
1 2 0.01135697635624036
1 3 -4.539830345783432e-05
1 4 0.35139348491986483
1 5 0.3186852206494785
1 6 -4.6807525552843494e-05
1 7 0.1445991435450918
1 8 0.17099915459530726
1 9 0.004035556367714974
1 10 -1.2077713083552299e-05
1 11 4

**Revisamos la ortonormalidad.** Asumiremos que los orbitales son ortonormales si $C^T SC$ difiere en menos de $10^{-6}$ respecto a la matriz identidad. Si los orbitales no son ortonormales, haremos hasta 3 intentos por ortonormalizarlos.

In [42]:
# Revisa ortonormalidad
orthonormality = True

CTSC = np.matmul(np.matmul(np.transpose(C),S),C)
ortho_deviation = np.abs(CTSC - np.identity(CTSC.shape[0]))

if (np.any(ortho_deviation > 10**-6)):
    orthonormality = False

if not orthonormality:
    print("Orthonormality violations {:d}, Maximum Violation {:f}".format((ortho_deviation > 10**-6).sum(),ortho_deviation.max()))        
else:
    print("No violations of the orthonormality")

No violations of the orthonormality


Se cambian de fase los obitales moleculares tal que **en cada orbital sea positivo el coeficiente de mayor magnitud**

In [43]:
# Vuelve positivo el elemento más largo de cada MO
for j in range(C.shape[1]):
    
    #Obtiene el índice del coeficiente con mayor valor absoluto del MO
    idxmaxabsval = 0
    for i in range(nbf):
        if(abs(C[i][j])>abs(C[idxmaxabsval][j])):
            idxmaxabsval = i
    
    # Ajusta el signo del MO
    sign = np.sign(C[idxmaxabsval][j])
    C[0:nbf,j] = sign*C[0:nbf,j]

In [44]:
nv = ncwo*ndoc

gamma = np.zeros((nbf5))
for i in range(ndoc):
    gamma[i] = np.arccos(np.sqrt(2.0*0.999-1.0))
    for j in range(ncwo-1):
        ig = ndoc+(i)*(ncwo-1)+j
        gamma[ig] = np.arcsin(np.sqrt(1.0/(ncwo-j)))        

In [45]:
iloop = 0
itlim = 1
E_old = E
sumdiff_old = 0

print('{:^7} {:^7} {:^14} {:^14} {:^14} {:^14}'.format("Nitext","Nitint","Eelec","Etot","Ediff","maxdiff"))

for i_ext in range(1000):
    
    #orboptr
    convgdelag,E_old,sumdiff_old,itlim,nzeros,fmiug0,C = orboptr(C,n,cj12,ck12,E_old,sumdiff_old,i_ext,itlim,nzeros,fmiug0)     
    
    #occopt
    gamma,elag,n,cj12,ck12 = occoptr(gamma,False,convgdelag,elag,C,H)
        
    if(convgdelag):
        break

Nitext  Nitint      Eelec           Etot          Ediff         maxdiff    
     1      0   -85.31637703   -75.96176633     0.13604936     0.06872477
     2     29   -85.38694312   -76.03233241    -0.01204467     0.05657592
     3     29   -85.39561133   -76.04100063    -0.00809231     0.01100368
     4     29   -85.40413831   -76.04952761    -0.00761352     0.01310715
     5     29   -85.41908782   -76.06447712    -0.01065344     0.01174037
     6     29   -85.42695651   -76.07234580    -0.00545494     0.00902169
     7     29   -85.42957742   -76.07496672    -0.00232732     0.00629730
     8     29   -85.43072302   -76.07611231    -0.00111928     0.00443317
     9     29   -85.43135963   -76.07674893    -0.00063554     0.00322738
    10     29   -85.43180350   -76.07719280    -0.00044123     0.00249022
    11     29   -85.43210961   -76.07749890    -0.00030393     0.00205823
    12     29   -85.43227932   -76.07766861    -0.00016850     0.00221293
    13     29   -85.43243333   -76.0

In [46]:
for i in range(C.shape[0]):
    print("------------------")
    for j in range(C.shape[0]):
        print(i,j,C[j][i])

------------------
0 0 1.0010234583482656
0 1 -0.006479412450788436
0 2 -0.010131219311472357
0 3 0.00044041563037283357
0 4 -5.040624642845394e-07
0 5 -0.0051318019633093855
0 6 0.0003451264357767571
0 7 -1.553007619860371e-07
0 8 -0.003191580843357785
0 9 -0.0013348039747312103
0 10 3.0619436456140455e-08
0 11 -2.2228457047976635e-05
0 12 -0.0011191435214533616
0 13 1.1223697993979267e-07
0 14 -0.0009881907213974731
0 15 -0.0017656260637589473
0 16 0.00033964831679148244
0 17 2.1591280640142497e-05
0 18 0.0008340625944105952
0 19 -0.0007744946955492851
0 20 -0.0017651298013485733
0 21 0.00033981335846919975
0 22 2.162959979515827e-05
0 23 -0.0008341485307703764
0 24 -0.0007744032661996816
------------------
1 0 -0.006236552350301106
1 1 -0.30253878302569576
1 2 -0.3726368278004996
1 3 -0.43283347271035494
1 4 2.4629705316813713e-05
1 5 -0.24088641019101742
1 6 -0.3423397330604077
1 7 9.093286957296301e-06
1 8 -0.19858512211189117
1 9 -0.0021487641403427123
1 10 1.124513520517817e-07
